In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!kaggle competitions download -c fake-news

 88% 41.0M/46.5M [00:00<00:00, 182MB/s] 
100% 46.5M/46.5M [00:00<00:00, 164MB/s]


In [4]:
import zipfile

zip_ref=zipfile.ZipFile('/content/fake-news.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import *

In [10]:
df=pd.read_csv('/content/train.csv')
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [11]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [12]:
df.dropna(inplace=True)

In [13]:
df.shape

(18285, 5)

In [14]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [16]:
X=df.drop(columns=['label'])
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [17]:
y=df['label']
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [19]:
X.shape,y.shape

((18285, 4), (18285,))

In [20]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [21]:
voc_size=10000

In [22]:
messages=X.copy()

In [23]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [25]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [26]:
messages.reset_index()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [27]:
import nltk
import re
from nltk.corpus import stopwords

In [28]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [39]:
import re
def remove_tags(text):
    cleaned_text=re.sub(re.compile('[^a-zA-Z]'),' ',text)
    return cleaned_text

In [40]:
messages['title']=messages['title'].apply(remove_tags)

In [41]:
messages

,id,title,author,text
0,0,House Dem Aide We Didn t Even See Comey s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,FLYNN Hillary Clinton Big Woman on Campus ...,Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T I Trump a Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,N F L Playoffs Schedule Matchups and Odds ...,Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,NATO Russia To Hold Parallel Exercises In Bal...,Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [29]:
from nltk.stem.porter import PorterStemmer

In [49]:
ps=PorterStemmer()
corpus=[]

for i in messages['title']:
  # review=re.sub('[^a-zA-Z]',' ',messages['title'][i])
  review=i
  review=review.lower()
  review=review.split()
  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

In [50]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [46]:
messages['title'][0]

'House Dem Aide  We Didn t Even See Comey s Letter Until Jason Chaffetz Tweeted It'

In [51]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [52]:
one_hot_repr=[one_hot(word,voc_size) for word in corpus]
one_hot_repr

[[2326, 2438, 2761, 609, 6183, 1799, 5619, 3904, 7684, 4505],
 [9036, 4628, 668, 8432, 1678, 6767, 4157],
 [7440, 6411, 1222, 6122],
 [7064, 4009, 9132, 5167, 7016, 358],
 [1702, 1678, 6652, 4534, 7567, 1786, 1678, 7605, 4476, 2221],
 [5237,
  8241,
  4426,
  7606,
  542,
  1630,
  4211,
  2802,
  1824,
  2020,
  6408,
  879,
  6384,
  5234,
  4157],
 [3516, 2437, 191, 8291, 3973, 2739, 9077, 7098, 391, 3389, 8325],
 [9393, 7755, 8081, 2225, 7812, 9546, 1630, 4407, 391, 3389, 8325],
 [3836, 7334, 4859, 8189, 9761, 3083, 5429, 4270, 1630, 2980],
 [4966, 8264, 4452, 3744, 3618, 197, 2477, 1445],
 [8653, 3862, 4030, 4091, 780, 5125, 556, 7341, 6931, 6797, 6155],
 [5167, 5289, 6183, 3083, 1630, 7812],
 [7218, 9269, 3055, 8509, 5973, 3301, 4280, 6848, 7800],
 [4387, 1333, 5207, 2352, 1553, 9004, 4154, 391, 3389, 8325],
 [5615, 6853, 2708, 9555, 8638, 391, 3389, 8325],
 [4263, 3583, 9681, 9363, 6718, 5511, 3662, 3284, 1808, 7775],
 [1084, 5137, 4628],
 [5791, 9863, 8206, 6127, 1630, 9715, 78

In [53]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [54]:
one_hot_repr[1]

[9036, 4628, 668, 8432, 1678, 6767, 4157]

In [55]:
sent_length=20
embedded_docs=pad_sequences(one_hot_repr,padding='pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 3904, 7684, 4505],
       [   0,    0,    0, ..., 1678, 6767, 4157],
       [   0,    0,    0, ..., 6411, 1222, 6122],
       ...,
       [   0,    0,    0, ...,  391, 3389, 8325],
       [   0,    0,    0, ..., 3458, 1235, 3194],
       [   0,    0,    0, ..., 6064, 7699, 3617]], dtype=int32)

In [56]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 9036, 4628,  668, 8432, 1678, 6767, 4157], dtype=int32)

In [83]:
model=Sequential([
    Embedding(voc_size,64,input_length=sent_length),
    LSTM(units=100),
    Dense(units=1,activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [58]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 64)            640000    
                                                                 
 lstm (LSTM)                 (None, 100)               66000     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 706,101
Trainable params: 706,101
Non-trainable params: 0
_________________________________________________________________


In [74]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [75]:
X_final

array([[   0,    0,    0, ..., 3904, 7684, 4505],
       [   0,    0,    0, ..., 1678, 6767, 4157],
       [   0,    0,    0, ..., 6411, 1222, 6122],
       ...,
       [   0,    0,    0, ...,  391, 3389, 8325],
       [   0,    0,    0, ..., 3458, 1235, 3194],
       [   0,    0,    0, ..., 6064, 7699, 3617]], dtype=int32)

In [76]:
y_final

array([1, 0, 1, ..., 0, 1, 1])

In [77]:
from sklearn.model_selection import train_test_split

In [78]:
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [84]:
history=model.fit(X_train,
          y_train,
          epochs=5,
          validation_data=(X_test,y_test),
          batch_size=32)

Epoch 1/5
383/383 [==============================] - 17s 37ms/step - loss: 0.2514 - accuracy: 0.8926 - val_loss: 0.1943 - val_accuracy: 0.9233
Epoch 2/5
383/383 [==============================] - 6s 16ms/step - loss: 0.1142 - accuracy: 0.9571 - val_loss: 0.2009 - val_accuracy: 0.9218
Epoch 3/5
383/383 [==============================] - 3s 8ms/step - loss: 0.0686 - accuracy: 0.9770 - val_loss: 0.2525 - val_accuracy: 0.9176
Epoch 4/5
383/383 [==============================] - 3s 7ms/step - loss: 0.0347 - accuracy: 0.9877 - val_loss: 0.2958 - val_accuracy: 0.9117
Epoch 5/5
383/383 [==============================] - 4s 11ms/step - loss: 0.0206 - accuracy: 0.9933 - val_loss: 0.3923 - val_accuracy: 0.9147


In [85]:
y_pred=model.predict(X_test)
y_pred=np.where(y_pred>0.5,1,0)

189/189 [==============================] - 1s 2ms/step


In [86]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [87]:
accuracy_score(y_test,y_pred)

0.9146644573322287

In [89]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.92      0.92      3419
           1       0.90      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



In [90]:
print(confusion_matrix(y_test,y_pred))

[[3141  278]
 [ 237 2379]]


In [123]:
class_names=['Fake','Not Fake']

In [175]:
def predict():

  input1=input('Enter your text: ')

  test_input=np.array(one_hot(input1,voc_size))
  test_input=pad_sequences([test_input],padding='pre',maxlen=sent_length)
  pred_prob=model.predict(test_input)
  pred=[1 if x>0.5 else 0 for x in pred_prob]
  pred_class=class_names[pred[0]]

  return pred_class

In [181]:
# a='how are you'
# onehot=one_hot(a,voc_size)
# b=pad_sequences([onehot],padding='pre',maxlen=20)
# pred_prob1=model.predict(b)
# print(pred_prob1)
# pred_class=[0 if x<0.5 else 1 for x in pred_prob1]
# print(pred_class)
# print(class_names[pred_class[0]])

In [174]:
predict()

Enter your text: Russian Researchers Discover Secret Nazi Military Base  Treasure Hunter  in the Arctic  Photos
1/1 [==============================] - 0s 356ms/step


'Not Fake'

In [148]:
X_test[9]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
       4741, 5819, 9389, 5024, 1103,  986, 9225, 7188, 4758], dtype=int32)

In [164]:
df[df['label']==1][['title','label']]

,title,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
2,Why the Truth Might Get You Fired,1
3,15 Civilians Killed In Single US Airstrike Hav...,1
4,Iranian woman jailed for fictional unpublished...,1
12,Russian Researchers Discover Secret Nazi Milit...,1
...,...,...
20788,Maine’s Gov. LePage Threatens To ‘Investigate’...,1
20791,Lawyer Who Kept Hillary Campaign Chief Out of ...,1
20793,Idiot Who Destroyed Trump Hollywood Star Gets ...,1
20798,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [165]:
messages['title'][12]

'Russian Researchers Discover Secret Nazi Military Base  Treasure Hunter  in the Arctic  Photos '

In [167]:
model.save('fake-news-detector.h5')

In [168]:
from keras.models import load_model

In [169]:
model2=load_model('fake-news-detector.h5')

In [180]:
model2.predict(tf.expand_dims(X_test[10],axis=0))

1/1 [==============================] - 0s 31ms/step


array([[1.5360816e-05]], dtype=float32)